In [10]:
import argparse
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col, count, collect_set, size, array, flatten, array_distinct, explode, rand, udf
from pyspark.sql.window import Window
import itertools

In [11]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

rdd_test = [("A", "Tonio", 4), ("B", "Tonio", 4), ("C", "Tonio", 4), ("D", "Tonio", 4),
            ("A", "Gatto", 4), ("B", "Gatto", 4), ("C", "Gatto", 4), ("D", "Gatto", 4), ("E", "Gatto", 4),
            ("A", "Chiara", 4), ("B", "Chiara", 4), ("C", "Chiara", 4), ("E", "Chiara", 4),
            ("A", "Daniele", 4), ("B", "Daniele", 4), ("C", "Daniele", 4), ("D", "Daniele", 4), ("E", "Daniele", 4),
            ("A", "Giulia", 4), ("B", "Giulia", 4), ("C", "Giulia", 4), ("D", "Giulia", 4), ("E", "Giulia", 4),
            ("A", "Giorgio", 4), ("B", "Giorgio", 4), ("C", "Giorgio", 4), ("D", "Giorgio", 4), ("E", "Giorgio", 4), ("F", "Giorgio", 4), ("G", "Giorgio", 4), ("H", "Giorgio", 4),
            ("A", "Giovanni", 4), ("B", "Giovanni", 4), ("C", "Giovanni", 4), ("D", "Giovanni", 4), ("E", "Giovanni", 4), ("F", "Giovanni", 4), ("G", "Giovanni", 4), ("H", "Giovanni", 4),
            ("A", "Giuseppe", 4), ("B", "Giuseppe", 4), ("C", "Giuseppe", 4), ("D", "Giuseppe", 4), ("E", "Giuseppe", 4), ("F", "Giuseppe", 4), ("G", "Giuseppe", 4), ("H", "Giuseppe", 4),
            ("A", "Giacomo", 4), ("B", "Giacomo", 4), ("C", "Giacomo", 4), ("D", "Giacomo", 4), ("E", "Giacomo", 4), ("F", "Giacomo", 4), ("G", "Giacomo", 4), ("H", "Giacomo", 4),
            ("A", "Gianluca", 4), ("B", "Gianluca", 4), ("C", "Gianluca", 4), ("D", "Gianluca", 4), ("E", "Gianluca", 4),
            ("A", "Gianmarco", 4), ("B", "Gianmarco", 4), ("C", "Gianmarco", 4), ("D", "Gianmarco", 4), ("E", "Gianmarco", 4),
            ("A", "Gianfranco", 4), ("B", "Gianfranco", 4), ("C", "Gianfranco", 4), ("D", "Gianfranco", 4), ("E", "Gianfranco", 4), ("F", "Gianfranco", 4), ("G", "Gianfranco", 4), ("H", "Gianfranco", 4),
            ("A", "Giancarlo", 4), ("B", "Giancarlo", 4), ("C", "Giancarlo", 4), ("D", "Giancarlo", 4), ("E", "Giancarlo", 4), ("F", "Giancarlo", 4), ("G", "Giancarlo", 4), ("H", "Giancarlo", 4)
        ]

rdd = [("A", "1", 4), ("C", "1", 4), ("B", "1", 4), ("C", "1", 2), ("D", "1", 4),
        ("B", "2", 4), ("C", "2", 4), ("A", "2", 4), 
        ("A", "3", 4), ("C", "3", 4),
        ("A", "4", 4), ("B", "4", 4), ("C", "4", 4),
        ("A", "5", 4), ("B", "5", 4), ("C", "5", 4),
        ("A", "6", 4), ("B", "6", 4), ("C", "6", 4), ("D", "6", 4), ("E", "6", 4), ("F", "6", 4),
        ("A", "7", 4), ("B", "7", 4), ("C", "7", 4),
        ("A", "8", 4), ("B", "8", 4), ("C", "8", 4),
        ("A", "9", 4), ("B", "9", 4), ("C", "9", 4),
        ("A", "10", 4), ("B", "10", 4), ("C", "10", 4),
        ("A", "11", 4), ("B", "11", 4), ("C", "11", 4), ("D", "11", 4),
        ("A", "12", 4), ("B", "12", 4), ("C", "12", 4), ("D", "12", 4),
        ("A", "13", 4), ("B", "13", 4), ("C", "13", 4), ("D", "13", 4), ("E", "13", 4),
        ("A", "14", 4), ("B", "14", 4), ("C", "14", 4), ("D", "14", 4), ("E", "14", 4),
        ("A", "15", 4), ("B", "15", 4), ("C", "15", 4), ("D", "15", 4), ("E", "15", 4),
        ("A", "16", 4), ("B", "16", 4), ("C", "16", 4), ("D", "16", 4), ("E", "16", 4), ("F", "16", 4)
        ]
schema = StructType([
    StructField("productId", StringType(), True),
    StructField("userId", StringType(), True),
    StructField("score", IntegerType(), True)
])

input_df = spark.createDataFrame(rdd_test, schema=schema).cache()

In [12]:
input_df.show()

+---------+-------+-----+
|productId| userId|score|
+---------+-------+-----+
|        A|  Tonio|    4|
|        B|  Tonio|    4|
|        C|  Tonio|    4|
|        D|  Tonio|    4|
|        A|  Gatto|    4|
|        B|  Gatto|    4|
|        C|  Gatto|    4|
|        D|  Gatto|    4|
|        E|  Gatto|    4|
|        A| Chiara|    4|
|        B| Chiara|    4|
|        C| Chiara|    4|
|        E| Chiara|    4|
|        A|Daniele|    4|
|        B|Daniele|    4|
|        C|Daniele|    4|
|        D|Daniele|    4|
|        E|Daniele|    4|
|        A| Giulia|    4|
|        B| Giulia|    4|
+---------+-------+-----+
only showing top 20 rows



In [13]:
input_df = input_df.filter(input_df["score"] >= 4)

In [14]:
joined_df = input_df.alias("df1").join(input_df.alias("df2"), ["productId"]) \
    .where(col("df1.userId") < col("df2.userId")) \
    .select(col("df1.userId").alias("userId1"), col("df2.userId").alias("userId2"), col("df1.productId").alias("productId")) \
    .distinct() 

joined_df.show()

+-------+----------+---------+
|userId1|   userId2|productId|
+-------+----------+---------+
|  Gatto|     Tonio|        A|
|  Gatto|    Giulia|        A|
|  Gatto|   Giorgio|        A|
|  Gatto|  Giovanni|        A|
|  Gatto|  Giuseppe|        A|
|  Gatto|   Giacomo|        A|
|  Gatto|  Gianluca|        A|
|  Gatto| Gianmarco|        A|
|  Gatto|Gianfranco|        A|
|  Gatto| Giancarlo|        A|
| Chiara|     Tonio|        A|
| Chiara|     Gatto|        A|
| Chiara|   Daniele|        A|
| Chiara|    Giulia|        A|
| Chiara|   Giorgio|        A|
| Chiara|  Giovanni|        A|
| Chiara|  Giuseppe|        A|
| Chiara|   Giacomo|        A|
| Chiara|  Gianluca|        A|
| Chiara| Gianmarco|        A|
+-------+----------+---------+
only showing top 20 rows



In [15]:
joined_df = input_df.alias("df1").join(input_df.alias("df2"), ["productId"]) \
    .where(col("df1.userId") < col("df2.userId")) \
    .select(col("df1.userId").alias("userId1"), col("df2.userId").alias("userId2"), col("df1.productId").alias("productId")) \

joined_df.show()

+-------+----------+---------+
|userId1|   userId2|productId|
+-------+----------+---------+
|  Gatto| Giancarlo|        A|
|  Gatto|Gianfranco|        A|
|  Gatto| Gianmarco|        A|
|  Gatto|  Gianluca|        A|
|  Gatto|   Giacomo|        A|
|  Gatto|  Giuseppe|        A|
|  Gatto|  Giovanni|        A|
|  Gatto|   Giorgio|        A|
|  Gatto|    Giulia|        A|
|  Gatto|     Tonio|        A|
|  Gatto| Giancarlo|        B|
|  Gatto|Gianfranco|        B|
|  Gatto| Gianmarco|        B|
|  Gatto|  Gianluca|        B|
|  Gatto|   Giacomo|        B|
|  Gatto|  Giuseppe|        B|
|  Gatto|  Giovanni|        B|
|  Gatto|   Giorgio|        B|
|  Gatto|    Giulia|        B|
|  Gatto|     Tonio|        B|
+-------+----------+---------+
only showing top 20 rows



In [16]:
from pyspark.sql.types import ArrayType, StringType


couple_df = joined_df.groupBy("userId1", "userId2").agg(collect_set("productId").alias("productsAffinity")) \
    .where(size(col("productsAffinity")) >= 3) \
    .select("userId1", "userId2", "productsAffinity") \
    .cache()

@udf(ArrayType(ArrayType(StringType())))
def get_combinations(values):
    list = []
    for i in range(3, len(values) + 1):
        list.extend(itertools.combinations(values, i))
    return list


couple_df = couple_df.withColumn("productsAffinity", explode(get_combinations(col("productsAffinity"))))

couple_df.show()

+---------+---------+----------------+
|  userId1|  userId2|productsAffinity|
+---------+---------+----------------+
|  Giacomo| Gianluca|       [C, E, B]|
|  Giacomo| Gianluca|       [C, E, A]|
|  Giacomo| Gianluca|       [C, E, D]|
|  Giacomo| Gianluca|       [C, B, A]|
|  Giacomo| Gianluca|       [C, B, D]|
|  Giacomo| Gianluca|       [C, A, D]|
|  Giacomo| Gianluca|       [E, B, A]|
|  Giacomo| Gianluca|       [E, B, D]|
|  Giacomo| Gianluca|       [E, A, D]|
|  Giacomo| Gianluca|       [B, A, D]|
|  Giacomo| Gianluca|    [C, E, B, A]|
|  Giacomo| Gianluca|    [C, E, B, D]|
|  Giacomo| Gianluca|    [C, E, A, D]|
|  Giacomo| Gianluca|    [C, B, A, D]|
|  Giacomo| Gianluca|    [E, B, A, D]|
|  Giacomo| Gianluca| [C, E, B, A, D]|
|Giancarlo|Gianmarco|       [C, E, B]|
|Giancarlo|Gianmarco|       [C, E, A]|
|Giancarlo|Gianmarco|       [C, E, D]|
|Giancarlo|Gianmarco|       [C, B, A]|
+---------+---------+----------------+
only showing top 20 rows



In [17]:
group_df = couple_df.withColumn(
    "userId",
    array("userId1", "userId2")
).groupBy("productsAffinity").agg(collect_set("userId").alias("groupUsers"))

group_df.show()

+------------------+--------------------+
|  productsAffinity|          groupUsers|
+------------------+--------------------+
|         [B, A, D]|[Gianluca, Giorgi...|
|         [C, A, D]|[Gianluca, Giorgi...|
|         [C, B, A]|[Gianluca, Giorgi...|
|      [C, B, A, D]|[Gianluca, Giorgi...|
|         [C, B, D]|[Gianluca, Giorgi...|
|         [C, E, A]|[Gianluca, Giorgi...|
|      [C, E, A, D]|[Gianluca, Giorgi...|
|         [C, E, B]|[Gianluca, Giorgi...|
|      [C, E, B, A]|[Gianluca, Giorgi...|
|   [C, E, B, A, D]|[Gianluca, Giorgi...|
|      [C, E, B, D]|[Gianluca, Giorgi...|
|         [C, E, D]|[Gianluca, Giorgi...|
|         [C, E, G]|[Giorgio, Giancar...|
|      [C, E, G, A]|[Giorgio, Giancar...|
|   [C, E, G, A, D]|[Giorgio, Giancar...|
|      [C, E, G, B]|[Giorgio, Giancar...|
|   [C, E, G, B, A]|[Giorgio, Giancar...|
|[C, E, G, B, A, D]|[Giorgio, Giancar...|
|   [C, E, G, B, D]|[Giorgio, Giancar...|
|      [C, E, G, D]|[Giorgio, Giancar...|
+------------------+--------------

In [18]:
group_df.orderBy(col("groupUsers")[0]).show()

shuffled_df = group_df.orderBy(rand(1))
shuffled_df.show()

shuffled_df.orderBy(col("groupUsers")[0]).show()

+----------------+--------------------+
|productsAffinity|          groupUsers|
+----------------+--------------------+
|       [B, A, D]|[Gianluca, Giorgi...|
|       [C, A, D]|[Gianluca, Giorgi...|
|       [C, B, A]|[Gianluca, Giorgi...|
|    [C, B, A, D]|[Gianluca, Giorgi...|
|       [C, B, D]|[Gianluca, Giorgi...|
|       [C, E, A]|[Gianluca, Giorgi...|
|    [C, E, A, D]|[Gianluca, Giorgi...|
|       [C, E, B]|[Gianluca, Giorgi...|
|    [C, E, B, A]|[Gianluca, Giorgi...|
| [C, E, B, A, D]|[Gianluca, Giorgi...|
|    [C, E, B, D]|[Gianluca, Giorgi...|
|       [C, E, D]|[Gianluca, Giorgi...|
|       [E, A, D]|[Gianluca, Giorgi...|
|       [E, B, A]|[Gianluca, Giorgi...|
|    [E, B, A, D]|[Gianluca, Giorgi...|
|       [E, B, D]|[Gianluca, Giorgi...|
|       [C, E, G]|[Giorgio, Giancar...|
|    [C, E, G, A]|[Giorgio, Giancar...|
| [C, E, G, A, D]|[Giorgio, Giancar...|
|    [C, E, G, B]|[Giorgio, Giancar...|
+----------------+--------------------+
only showing top 20 rows



+------------------+--------------------+
|  productsAffinity|          groupUsers|
+------------------+--------------------+
|[F, C, E, B, A, D]|[Giorgio, Giancar...|
|   [F, C, E, B, A]|[Giorgio, Giancar...|
|      [F, G, H, A]|[Giorgio, Giancar...|
|         [H, B, A]|[Giorgio, Giancar...|
|   [F, E, H, B, A]|[Giorgio, Giancar...|
|      [F, E, B, D]|[Giorgio, Giancar...|
|         [F, B, D]|[Giorgio, Giancar...|
|      [F, C, H, B]|[Giorgio, Giancar...|
|   [F, C, H, A, D]|[Giorgio, Giancar...|
|   [C, E, G, B, D]|[Giorgio, Giancar...|
|      [C, E, G, H]|[Giorgio, Giancar...|
|[F, E, G, H, B, D]|[Giorgio, Giancar...|
|      [E, B, A, D]|[Gianluca, Giorgi...|
|   [F, E, G, H, B]|[Giorgio, Giancar...|
|   [F, C, E, H, B]|[Giorgio, Giancar...|
|      [F, E, G, B]|[Giorgio, Giancar...|
|      [C, B, A, D]|[Gianluca, Giorgi...|
|   [F, G, B, A, D]|[Giorgio, Giancar...|
|      [C, G, H, D]|[Giorgio, Giancar...|
|[C, E, G, H, B, A]|[Giorgio, Giancar...|
+------------------+--------------